In [1]:
from sqlalchemy import create_engine

import pymysql

import pandas as pd

import hashlib

import base64

import pycountry_convert as pc

import numpy as np

include libraries

In [2]:
oldDb_username = 'root'
oldDb_password = 'root'
oldDb_host     = '127.0.0.1'
oldDb_port     = '8889'
oldDb_database = 'StudDB 2.0'

In [3]:
arcDb_username = 'root'
arcDb_password = 'root'
arcDb_host     = '127.0.0.1'
arcDb_port     = '8889'
arcDb_database = 'archivDb'

In [4]:
sqlEngine = create_engine('mysql+pymysql://' + oldDb_username + ':' + oldDb_password + '@' + oldDb_host + ':' + oldDb_port + '/' + oldDb_database, pool_recycle=3600)

dbConnection = sqlEngine.connect()


 

pd.set_option('display.expand_frame_repr', False) 

establish database connection

In [5]:
def hashCol (col):
    return col.astype(str).str.encode('UTF-8')\
          .apply(lambda x: base64.b64encode(hashlib.sha1(x.lower()).digest()))

create hashing function

In [6]:
def countryMapper (x):
    continents = {
        'NA': 'North America',
        'SA': 'South America', 
        'AS': 'Asia',
        'OC': 'Oceania',
        'AF': 'Africa',
        'EU': 'Europe'
        }
    return continents[pc.country_alpha2_to_continent_code( pc.country_name_to_country_alpha2(x, cn_name_format="default") )]

def mapCountry (col):
    return col.astype(str)\
          .apply(countryMapper)

create country mapper to continent

In [7]:
query = """SELECT
  `user`.`firstname` AS 'student_firstname',
  `user`.`lastname`  AS 'student_lastname',
  YEAR(`student`.`birthdate`) AS 'birth_year',
  IF(`user`.`salutation_id` = 1, 'Female',
          IF(`user`.`salutation_id` = 2, 'Male', NULL)
    ) AS 'salutation',
  `country`.`name` AS 'nationality',

  `home_university`.`abbreviation` AS 'home_university',
  `home_degree`.`abbreviation` AS 'home_degree',
  `home_course`.`name` AS 'home_program',
  `study_home`.`home_semester` AS 'semester',

  `host_university`.`abbreviation` AS 'abroad_university',
  if(`application`.`intention_id` = 1, 'Exchange', 'Double Degree') AS 'type_of_transfer',
  `host_degree`.`abbreviation` AS 'abroad_degree',
  `host_course`.`name` AS 'abroad_program',

  IF(`student`.`student_status_id` = 4, 'Alumni', 'Dropout') AS 'overall_status',
  `exchange_period`.`exchange_semester` AS 'start_of_transfer',
  `study_home`.`home_cgpa` AS 'average_grade',
  `study_host`.`foreign_num_planed_exams` AS 'number_planned_exams',

  `study_host`.`foreign_matno` AS 'abroad_mat_no',
  `exchange_period`.`semester_end` AS 'exchange_semester_end',
  `exchange_period`.`semester_begin` AS 'exchange_semester_start'
FROM `student`

# Join for user related datas
JOIN `user`
  ON `user`.`user_id` = `student`.`user_id`

JOIN `country`
  ON `country`.`country_id` = `student`.`nationality_country_id`

JOIN `application`
  ON `application`.`student_id` = `student`.`student_id`

# Join for home study datas
JOIN `study_home`
  ON `study_home`.`application_id` = `application`.`application_id`

JOIN `university` AS `home_university`
  ON `home_university`.`university_id` = `study_home`.`home_university_id`

JOIN `degree` AS `home_degree`
  ON `home_degree`.`degree_id` = `study_home`.`home_degree_id`

JOIN `course` AS `home_course`
  ON `home_course`.`course_id` = `study_home`.`home_course_id`

# Join for foreign studies
LEFT JOIN `exchange`
  ON `exchange`.`application_id` = `application`.`application_id`

LEFT JOIN `study_host`
  ON `study_host`.`exchange_id` = `exchange`.`exchange_id`

LEFT JOIN `university` AS `host_university`
  ON `host_university`.`university_id` = `study_host`.`foreign_uni_id`

LEFT JOIN `degree` AS `host_degree`
  ON `host_degree`.`degree_id` = `study_host`.`foreign_degree_id`

LEFT JOIN `course` AS `host_course`
  ON `host_course`.`course_id` = `study_host`.`foreign_course_id`


LEFT JOIN `exchange_period`
  ON `exchange_period`.`period_id` = `application`.`exchange_period_id`

WHERE
  `student`.`student_status_id` >= 4"""

retrieve from StudDB 2.0 (status= alumni or dropout) with all of the corresponding data

In [8]:
frame = pd.read_sql(query, dbConnection);
cols = ['student_firstname','student_lastname','abroad_mat_no']
for col in cols:
    frame[col] = hashCol(frame[col])
frame['nationality'] = mapCountry(frame['nationality'])
frame['exchange_semester_start'] =  pd.to_datetime(frame['exchange_semester_start'], format='%Y-%m-%d')
frame['exchange_semester_end'] =  pd.to_datetime(frame['exchange_semester_end'], format='%Y-%m-%d')
frame['study_time_months'] = (frame['exchange_semester_end'] - frame['exchange_semester_start']).dt.days/30.4
frame = frame.drop(['exchange_semester_start', 'exchange_semester_end'], axis=1)
frame.head(10)

,student_firstname,student_lastname,birth_year,salutation,nationality,home_university,home_degree,home_program,semester,abroad_university,type_of_transfer,abroad_degree,abroad_program,overall_status,start_of_transfer,average_grade,number_planned_exams,abroad_mat_no,study_time_months
0,b'/gm8LvJzejJY+XjiYibcusGz+Ug=',b'Wvjx6d246a+4DIDt1ocMyx8J/oo=',1992,Male,South America,UDE,B.Sc.,Computer Engineering,8,ZZULI,Exchange,B.Sc.,Computer Engineering,Dropout,SS2021,3.7,10,b'hjGdEUtsDkGRB0+dpdEYvmJhr68=',7.006579
1,b'MrNJEzZSLgc0iXJbXa8pjNdJAHo=',b'Mo45Y4ExPBYgfrvjvuQ2OH19/D0=',1995,Female,Asia,NTU,B.Sc.,Computer Engineering,7,UDE,Exchange,B.Sc.,Computer Engineering,Alumni,None,1.7,0,b'mYK/ISEtR/+4A0ghG6JGAGjcwx0=',NaN


anonymize and process the data for archival

In [9]:
dbConnection.close()

close connection to StudDB 2.0

In [10]:
sqlEngine = create_engine('mysql+pymysql://' + arcDb_username + ':' + arcDb_password + '@' + arcDb_host + ':' + arcDb_port + '/' + arcDb_database, pool_recycle=3600)

dbConnection = sqlEngine.connect()

establish connection with archivDB

In [11]:
try:

    frame.to_sql('inactive_students', dbConnection, if_exists='append', index=False);

except ValueError as vx:

    print(vx)

except Exception as ex:   

    print(ex) 

finally:

    dbConnection.close()

place the processed data into the archivDB in inactive_students table

In [12]:
sqlEngine = create_engine('mysql+pymysql://' + oldDb_username + ':' + oldDb_password + '@' + oldDb_host + ':' + oldDb_port + '/' + oldDb_database, pool_recycle=3600)

dbConnection = sqlEngine.connect()

# Cleanup script
dbConnection.execute('DELETE FROM `student` WHERE `student`.`student_status_id` >= 4')
dbConnection.close()

cleanup the script & drop corresponding data